# L3 M11.1: Multi-Tenant RAG Architecture Patterns

## Learning Arc

**Purpose:** Build production-ready multi-tenant RAG systems that reduce infrastructure costs from ₹29 crore to ₹5.8 crore annually while maintaining complete data isolation across 50+ business units.

**Concepts Covered:**
- Isolation Models (Shared-DB, Shared-Schema, Separate-DB, Hybrid)
- Tenant Routing Strategies (JWT claims, API key parsing, subdomain-based)
- Context Propagation (async-safe using Python contextvars)
- Vector Database Namespacing (Pinecone namespaces, Qdrant collections)
- PostgreSQL Row-Level Security (RLS) for metadata isolation
- Per-Tenant Rate Limiting and quota enforcement
- Cost Attribution and metering for accurate chargeback
- GDPR Compliance (soft-delete, 90-day retention, complete purge)
- Cross-Tenant Leak Prevention (automated testing, continuous monitoring)

**After Completing This Notebook:**
- You will understand the four isolation models and their cost/security tradeoffs
- You can design tenant routing architecture with FastAPI middleware
- You will quantify isolation costs at 10, 50, 100 tenant scales
- You can identify when NOT to use multi-tenancy (classified data, banking, healthcare)
- You will implement namespace-based vector isolation preventing data leaks
- You can configure PostgreSQL RLS policies for defense-in-depth
- You will build per-tenant rate limiting and cost tracking

**Context in Track L3.M11:**
This module builds on single-tenant RAG implementations (M1-M8) and prepares you for multi-tenant monitoring and cost attribution (M11.2).

In [ ]:
# Cell 2: Environment Setup
import os
import sys

# Add src to path for imports
if '../src' not in sys.path:
    sys.path.insert(0, '../src')
if '..' not in sys.path:
    sys.path.insert(0, '..')

# OFFLINE mode for L3 consistency
OFFLINE = os.getenv("OFFLINE", "false").lower() == "true"

# Service detection from script - PINECONE (vector DB) + OPENAI (embeddings/LLM)
PINECONE_ENABLED = os.getenv("PINECONE_ENABLED", "false").lower() == "true"
OPENAI_ENABLED = os.getenv("OPENAI_ENABLED", "false").lower() == "true"

if OFFLINE or (not PINECONE_ENABLED and not OPENAI_ENABLED):
    print("⚠️  Running in OFFLINE/SERVICE_DISABLED mode")
    print("   → External API calls will be skipped")
    print("   → Set PINECONE_ENABLED=true and OPENAI_ENABLED=true in .env to enable")
    print("   → Notebook demonstrates architecture patterns using in-memory storage")
else:
    print("✓ Online mode - external services enabled")
    if PINECONE_ENABLED:
        print("  ✓ Pinecone vector database")
    if OPENAI_ENABLED:
        print("  ✓ OpenAI embeddings and LLM")

## Section 1: Introduction & Hook - The Cost Problem

**Business Case:** A GCC serving 50 business units faces a decision:
- **Separate Systems:** ₹29 crore annually (₹25Cr infrastructure + ₹4Cr operations)
- **Shared Infrastructure:** ₹5.8 crore annually (₹5Cr infrastructure + ₹80L operations)
- **Savings:** ₹23.2 crore (80% reduction)

**The Challenge:** How do you share infrastructure while maintaining complete data isolation, regulatory compliance, and independent SLAs?

In [ ]:
# Cell 4: Cost Comparison Visualization
import json

# Load example data
with open('../example_data.json', 'r') as f:
    data = json.load(f)

cost_analysis = data['cost_analysis_50_tenants']

print("Cost Analysis for 50 Tenants:\n")
for model, costs in cost_analysis.items():
    print(f"{model.replace('_', ' ').title()}:")
    print(f"  Total: {costs['total']}")
    print(f"  Per Tenant: {costs['per_tenant']}")
    if 'savings_vs_separate' in costs:
        print(f"  Savings: {costs['savings_vs_separate']}")
    print()

# Expected output:
# Separate Systems: ₹29 Cr total, ₹58L per tenant
# Shared DB: ₹5.8 Cr total, ₹11.6L per tenant, 80% savings
# Hybrid: ₹11.2 Cr total, ₹22.4L per tenant, 61% savings

## Section 2: Conceptual Foundation - Isolation Spectrum

### Four Isolation Models

**1. Shared-DB (Logical Isolation):**
- One PostgreSQL, one vector DB
- Isolation via `WHERE tenant_id = 'finance'` filters and vector namespaces
- Fast onboarding (15 min), lowest cost (₹11.6L/tenant)
- Risk: Noisy neighbor, cross-tenant leak if filter forgotten

**2. Shared-Schema (Row-Level Security):**
- Tenant_id column with PostgreSQL RLS policies
- Automatic filtering at database level (defense-in-depth)
- Medium cost (₹13.6L/tenant), 30 min onboarding
- Risk: Still shares compute/memory resources

**3. Separate-DB (Physical Isolation):**
- Dedicated PostgreSQL and vector DB per tenant
- True isolation, no leak risk, custom schemas possible
- High cost (₹58L/tenant), slow onboarding (2 weeks)
- Use case: Banking, healthcare, classified data

**4. Hybrid (80/20 Model):**
- 80% tenants use Shared-DB (standard security)
- 20% tenants use Separate-DB (high security/regulatory)
- Balanced cost (₹22.4L/tenant average), flexible onboarding
- **GCC Production Recommendation**

In [ ]:
# Cell 6: Isolation Models Comparison
isolation_models = data['isolation_models_comparison']

print("Isolation Models Comparison:\n")
for model, details in isolation_models.items():
    print(f"{model.replace('_', ' ').upper()}:")
    print(f"  Description: {details['description']}")
    print(f"  Cost/Tenant/Year: {details['cost_per_tenant_per_year']}")
    print(f"  Onboarding Time: {details['onboarding_time']}")
    print(f"  Use Case: {details['use_case']}")
    print()

# Expected: 4 isolation models with cost and onboarding details

## Section 3: Technology Stack

**API Layer:** FastAPI (async-native, built-in dependency injection)

**Auth:** JWT with tenant_id claims (Auth0/Keycloak) or API key parsing

**Context Propagation:** Python contextvars (thread-safe, async-safe)

**Metadata DB:** PostgreSQL with Row-Level Security (RLS)

**Vector DB Options:**
- **Pinecone:** Namespace-based isolation (cost-efficient)
- **Qdrant:** Collection-per-tenant (dedicated resources)

**Cache:** Redis with tenant-aware keys (`{tenant_id}:resource:id`)

**Monitoring:** Prometheus with tenant labels, Grafana per-tenant dashboards

In [ ]:
# Cell 8: Import Core Module
from l3_m11_multi_tenant_foundations import (
    TenantRegistry,
    TenantContextManager,
    VectorDBIsolation,
    create_tenant,
    get_tenant,
    set_current_tenant,
    get_current_tenant,
)

print("✓ Multi-tenant module imported successfully")
print("  Available classes: TenantRegistry, TenantContextManager, VectorDBIsolation")
print("  Available functions: create_tenant, get_tenant, set_current_tenant, get_current_tenant")

## Section 4: Working Implementation - Tenant CRUD

### Creating Tenants with Tier-Based Limits

In [ ]:
# Cell 10: Create Tenant Registry and Tenants
# Initialize registry
registry = TenantRegistry()
vector_db = VectorDBIsolation()

# Create Finance tenant (Premium tier)
finance = registry.create_tenant(
    tenant_id='finance',
    tenant_name='Finance Department',
    tier='premium',
    isolation_model='shared-db',
    admin_email='admin@finance.example.com'
)

print("Created Finance Tenant:")
print(f"  ID: {finance['tenant_id']}")
print(f"  Tier: {finance['tier']}")
print(f"  Status: {finance['status']}")
print(f"  Isolation: {finance['isolation_model']}")

# Check tier limits
limits = registry.limits['finance']
print(f"\nPremium Tier Limits:")
print(f"  Max Queries/Day: {limits['max_queries_per_day']:,}")
print(f"  Rate Limit: {limits['rate_limit_rpm']} RPM")

# Expected:
# Finance tenant created with premium tier
# Limits: 100,000 queries/day, 1,000 RPM

In [ ]:
# Cell 11: Create Legal Tenant (Standard Tier)
legal = registry.create_tenant(
    tenant_id='legal',
    tenant_name='Legal Department',
    tier='standard',
    isolation_model='shared-db',
    admin_email='admin@legal.example.com'
)

print("Created Legal Tenant:")
print(f"  ID: {legal['tenant_id']}")
print(f"  Tier: {legal['tier']}")

# Compare limits
standard_limits = registry.limits['legal']
print(f"\nStandard vs Premium Limits:")
print(f"  Standard Queries/Day: {standard_limits['max_queries_per_day']:,}")
print(f"  Premium Queries/Day: {limits['max_queries_per_day']:,}")
print(f"  Difference: {limits['max_queries_per_day'] / standard_limits['max_queries_per_day']}x")

# Expected:
# Standard: 10,000 queries/day
# Premium: 100,000 queries/day (10x difference)

### Tenant Context Propagation

**Challenge:** How does `tenant_id` flow through async operations without manual parameter passing?

**Solution:** Python's `contextvars` module provides async-safe storage that propagates through:
- FastAPI request handlers
- Async function calls
- Background tasks (with explicit passing)
- Celery workers (requires manual propagation)

In [ ]:
# Cell 13: Context Propagation Demo
# Set tenant context
set_current_tenant('finance')

# Retrieve from context (no parameter passing needed)
current_tenant = get_current_tenant()
print(f"Current tenant context: {current_tenant}")

# Simulate async call chain
async def level_1_function():
    tenant = get_current_tenant()
    print(f"  Level 1 sees tenant: {tenant}")
    await level_2_function()

async def level_2_function():
    tenant = get_current_tenant()
    print(f"    Level 2 sees tenant: {tenant}")

import asyncio
await level_1_function()

# Expected:
# Current tenant context: finance
# Level 1 sees tenant: finance
# Level 2 sees tenant: finance

### Vector Database Namespace Isolation

In [ ]:
# Cell 15: Provision Vector Namespaces
# Provision namespaces for both tenants
vector_db.provision_tenant_namespace('finance')
vector_db.provision_tenant_namespace('legal')

print("Vector Namespaces Provisioned:")
print(f"  Finance: {vector_db.get_tenant_namespace('finance')}")
print(f"  Legal: {vector_db.get_tenant_namespace('legal')}")

# Check isolation
print(f"\nNamespace Count: {len(vector_db.vectors)}")
print(f"Namespaces: {list(vector_db.vectors.keys())}")

# Expected:
# Finance: tenant_finance
# Legal: tenant_legal
# 2 separate namespaces created

In [ ]:
# Cell 16: Ingest Documents with Tenant Isolation
# Finance documents
finance_docs = [
    {
        'id': 'finance-q4-2024',
        'embedding': [0.1] * 1536,  # Placeholder vector
        'metadata': {
            'title': 'Q4 Financial Report 2024',
            'content': 'Revenue exceeded targets by 15%',
            'classification': 'confidential'
        }
    },
    {
        'id': 'finance-audit-2024',
        'embedding': [0.2] * 1536,
        'metadata': {
            'title': 'Audit Findings 2024',
            'content': '3 minor findings identified',
            'classification': 'restricted'
        }
    }
]

vector_db.upsert_documents_for_tenant('finance', finance_docs)
print("Finance Documents Ingested: 2")

# Legal documents
legal_docs = [
    {
        'id': 'legal-gdpr-guide',
        'embedding': [0.3] * 1536,
        'metadata': {
            'title': 'GDPR Compliance Guide',
            'content': 'Data retention requirements',
            'classification': 'internal'
        }
    }
]

vector_db.upsert_documents_for_tenant('legal', legal_docs)
print("Legal Documents Ingested: 1")

# Verify isolation
finance_ns = vector_db.get_tenant_namespace('finance')
legal_ns = vector_db.get_tenant_namespace('legal')

print(f"\nDocument Counts:")
print(f"  Finance: {vector_db.vectors[finance_ns]['metadata']['document_count']}")
print(f"  Legal: {vector_db.vectors[legal_ns]['metadata']['document_count']}")

# Expected:
# Finance: 2 documents
# Legal: 1 document
# Documents stored in separate namespaces

### Cross-Tenant Isolation Testing

In [ ]:
# Cell 18: Test Cross-Tenant Isolation
# Query Finance namespace
finance_results = vector_db.query_vectors_for_tenant(
    tenant_id='finance',
    query_embedding=[0.1] * 1536,
    top_k=10
)

print("Finance Query Results:")
for i, result in enumerate(finance_results, 1):
    print(f"  {i}. {result['metadata']['title']}")
    print(f"     Tenant: {result['metadata']['tenant_id']}")

# Query Legal namespace
legal_results = vector_db.query_vectors_for_tenant(
    tenant_id='legal',
    query_embedding=[0.3] * 1536,
    top_k=10
)

print("\nLegal Query Results:")
for i, result in enumerate(legal_results, 1):
    print(f"  {i}. {result['metadata']['title']}")
    print(f"     Tenant: {result['metadata']['tenant_id']}")

# Verify no cross-tenant results
finance_tenant_ids = {r['metadata']['tenant_id'] for r in finance_results}
legal_tenant_ids = {r['metadata']['tenant_id'] for r in legal_results}

print(f"\nIsolation Verification:")
print(f"  Finance sees only 'finance': {finance_tenant_ids == {'finance'}}")
print(f"  Legal sees only 'legal': {legal_tenant_ids == {'legal'}}")

# Expected:
# Finance query returns 2 documents (Q4 Report, Audit Findings)
# Legal query returns 1 document (GDPR Guide)
# No cross-tenant results (isolation verified)

## Section 5: Reality Check - Common Pitfalls

### What Can Go Wrong in Production

In [ ]:
# Cell 20: Simulate Failure - Forgetting Namespace Parameter
print("Failure Scenario: Developer forgets namespace parameter\n")

# WRONG: Query without namespace (simulated)
# In production, this would return ALL tenants' data
print("❌ BAD CODE:")
print("   index.query(vector=query_embedding, top_k=5)")
print("   # Missing namespace parameter!")
print("   # Returns: Finance, Legal, HR documents (CROSS-TENANT LEAK)\n")

# CORRECT: Always specify namespace
print("✓ CORRECT CODE:")
print("   index.query(")
print("       vector=query_embedding,")
print("       namespace=f'tenant_{tenant_id}',  # CRITICAL")
print("       top_k=5")
print("   )")
print("   # Returns: Only Finance documents\n")

# FIX: Create wrapper function
print("BEST PRACTICE: Wrapper function")
print("   def query_vectors_for_tenant(tenant_id, embedding, top_k=5):")
print("       namespace = f'tenant_{tenant_id}'")
print("       results = index.query(vector=embedding, namespace=namespace, top_k=top_k)")
print("       # Validate results")
print("       for r in results:")
print("           assert r.metadata['tenant_id'] == tenant_id")
print("       return results")

In [ ]:
# Cell 21: Failure - Cache Poisoning
print("Failure Scenario: Cache key without tenant_id\n")

# WRONG: Cache key without tenant isolation
print("❌ BAD CODE:")
print("   cache_key = f'embedding:{query_hash}'")
print("   # Finance and Legal both use same cached embedding!\n")

# CORRECT: Tenant-aware cache key
print("✓ CORRECT CODE:")
print("   cache_key = f'{tenant_id}:embedding:{query_hash}'")
print("   # Finance: 'finance:embedding:abc123'")
print("   # Legal: 'legal:embedding:abc123'")
print("   # Separate cache entries per tenant")

## Section 6: Alternative Solutions

### Qdrant Collections vs Pinecone Namespaces

**Pinecone Namespaces:**
- Single index, multiple namespaces
- Cost-efficient (shared index infrastructure)
- Namespace parameter required on every query
- Risk: Forgetting namespace returns all data

**Qdrant Collections:**
- Dedicated collection per tenant
- Physical isolation (separate storage)
- Collection name in query (harder to forget)
- Higher cost (more infrastructure)

In [ ]:
# Cell 23: Alternative - Qdrant Pattern (Conceptual)
print("Qdrant Collection-Per-Tenant Pattern:\n")

print("# Create dedicated collection for each tenant")
print("qdrant.create_collection(")
print("    collection_name='tenant_finance',")
print("    vectors_config={'size': 1536, 'distance': 'Cosine'}")
print(")\n")

print("# Query specific tenant collection")
print("results = qdrant.search(")
print("    collection_name='tenant_finance',  # Explicit tenant")
print("    query_vector=embedding,")
print("    limit=5")
print(")\n")

print("Advantages:")
print("  ✓ Physical isolation (separate storage/indices)")
print("  ✓ Collection name explicit (harder to leak)")
print("  ✓ Per-tenant scaling (resize individual collections)\n")

print("Disadvantages:")
print("  ✗ Higher cost (50 collections = 50× infrastructure)")
print("  ✗ Operational overhead (manage 50+ collections)")
print("  ✗ Slower onboarding (collection creation)")

## Section 7: When NOT to Use Multi-Tenancy

**Do NOT use multi-tenancy if:**

1. **<10 tenants** - Operational complexity exceeds cost savings
2. **Banking/Financial Services** - Customer account data requires isolated infrastructure (regulatory)
3. **Healthcare** - Patient records require HIPAA-compliant isolation
4. **Government/Classified Data** - Top Secret/classified requires physical separation
5. **Vastly Different Customization** - Custom models, unique pipelines per tenant

**Use separate systems when:**
- Regulatory prohibition on shared infrastructure
- Data sovereignty requires in-country deployment
- SLA requirements differ drastically (99.999% vs 99.5%)
- Customer contract mandates dedicated resources

In [ ]:
# Cell 25: Decision Framework
print("Decision Framework: Multi-Tenant vs Separate Systems\n")

scenarios = [
    {
        'scenario': 'GCC serving 50 business units (Finance, HR, Legal, etc.)',
        'tenant_count': 50,
        'data_type': 'Internal corporate documents',
        'compliance': 'SOX, GDPR (non-overlapping)',
        'recommendation': 'HYBRID (80% Shared-DB, 20% Separate-DB)',
        'reasoning': 'Cost savings (61%), flexible security tiers'
    },
    {
        'scenario': 'Bank serving retail customers',
        'tenant_count': 1000000,
        'data_type': 'Customer account data (PII)',
        'compliance': 'PCI-DSS, banking regulations',
        'recommendation': 'SEPARATE SYSTEMS (per customer isolation)',
        'reasoning': 'Regulatory requirement for isolation'
    },
    {
        'scenario': 'Healthcare RAG for 5 hospitals',
        'tenant_count': 5,
        'data_type': 'Patient medical records',
        'compliance': 'HIPAA',
        'recommendation': 'SEPARATE SYSTEMS',
        'reasoning': 'HIPAA requires isolated infrastructure, <10 tenants'
    },
    {
        'scenario': 'Startup with 3 enterprise clients',
        'tenant_count': 3,
        'data_type': 'Enterprise documents',
        'compliance': 'Standard NDA',
        'recommendation': 'SEPARATE SYSTEMS',
        'reasoning': '<10 tenants - multi-tenancy overhead not justified'
    }
]

for s in scenarios:
    print(f"Scenario: {s['scenario']}")
    print(f"  Tenants: {s['tenant_count']}")
    print(f"  Data: {s['data_type']}")
    print(f"  Compliance: {s['compliance']}")
    print(f"  → Recommendation: {s['recommendation']}")
    print(f"  → Reasoning: {s['reasoning']}")
    print()

## Section 8: Common Failures - Production Incidents

### Real-World Failure Modes and Fixes

In [ ]:
# Cell 27: Common Failures Table
import pandas as pd

failures = [
    {
        'failure': 'Context Propagation Breaks in Background Tasks',
        'cause': 'Celery workers don\'t inherit async context from HTTP request',
        'impact': 'Documents indexed to wrong namespace, documents unsearchable',
        'fix': 'Pass tenant_id explicitly to Celery tasks, set context in worker'
    },
    {
        'failure': 'Forgetting Namespace in Vector Query',
        'cause': 'Developer omits namespace parameter in index.query()',
        'impact': 'Cross-tenant data leak, Tenant A retrieves Tenant B documents',
        'fix': 'Create wrapper function that ALWAYS adds namespace, add assertion'
    },
    {
        'failure': 'PostgreSQL RLS Policy Disabled During Migration',
        'cause': 'DBA disables RLS for bulk updates, forgets to re-enable',
        'impact': '3+ hours of all tenants accessing all data, regulatory incident',
        'fix': 'Wrap enable/disable in same transaction, daily monitoring check'
    },
    {
        'failure': 'Cache Poisoning Across Tenants',
        'cause': 'Cache key doesn\'t include tenant_id',
        'impact': 'Finance and Legal retrieve each other\'s embeddings/query results',
        'fix': 'Include tenant_id as first cache key segment: {tenant_id}:embedding:{query}'
    },
    {
        'failure': 'Rate Limiting Shared Across Tenants',
        'cause': 'Global rate limiter tracks requests for all tenants',
        'impact': 'One tenant exhausts quota, blocks all other 49 tenants',
        'fix': 'Implement per-tenant rate limiting with Redis: rate_limit:{tenant_id}:rpm'
    }
]

df = pd.DataFrame(failures)
print("Common Multi-Tenant Failures:\n")
for i, row in df.iterrows():
    print(f"{i+1}. {row['failure']}")
    print(f"   Cause: {row['cause']}")
    print(f"   Impact: {row['impact']}")
    print(f"   Fix: {row['fix']}")
    print()

## Section 9: GCC Enterprise Context

### Production Deployment Checklist

In [ ]:
# Cell 29: Production Checklist
checklist = [
    "✓ Tenant isolation validated via external penetration testing",
    "✓ Cross-tenant leak detection automated (runs on every deployment)",
    "✓ Cost tracking accurate to ±2% (CFO chargeback validation)",
    "✓ Multi-tenant monitoring dashboard operational (per-tenant metrics)",
    "✓ Tenant onboarding automated (<1 day for standard tier)",
    "✓ Blast radius containment verified (one tenant's failure isolated)",
    "✓ SLA monitoring per tenant (99.9% premium, 99.5% standard)",
    "✓ Audit trail comprehensive (every query logged with tenant_id)"
]

print("GCC Production Success Criteria:\n")
for item in checklist:
    print(f"  {item}")

print("\nDeployment Warnings:\n")
print("  ⚠️  Multi-tenancy requires rigorous isolation testing")
print("  ⚠️  Single cross-tenant leak violates SOX/GDPR compliance")
print("  ⚠️  Implement continuous validation (automated tests + quarterly pentest)")
print("  ⚠️  Scaling ceiling: 50 tenants for Shared-DB (connection pool limit)")
print("  ⚠️  At 100 tenants, shard across multiple databases (3-month refactor)")

## Summary: Key Takeaways

**What You Learned:**
1. **Four isolation models** with cost/security tradeoffs (Shared-DB saves 80%, Separate-DB required for banking/healthcare)
2. **Tenant routing middleware** extracts tenant_id from JWT/API keys, validates status, sets async context
3. **Context propagation** using contextvars ensures tenant_id flows through async calls without parameter passing
4. **Vector namespace isolation** prevents cross-tenant leaks (CRITICAL: always specify namespace parameter)
5. **PostgreSQL RLS** provides defense-in-depth with automatic tenant filtering
6. **Per-tenant rate limiting** prevents noisy neighbor problems
7. **Common failures** and fixes (forgetting namespace, cache poisoning, RLS disabled)
8. **When NOT to use** multi-tenancy (<10 tenants, banking/healthcare, classified data)

**Production Recommendations:**
- **Hybrid model (80/20)** for GCC deployments (61% cost savings vs separate systems)
- **Automated testing** for cross-tenant isolation (run on every deployment)
- **Wrapper functions** to enforce namespace parameters
- **Continuous monitoring** for RLS status, cache isolation, rate limits

**Next Steps:**
- Implement multi-tenant monitoring and cost attribution (L3 M11.2)
- Deploy automated isolation testing (quarterly penetration tests)
- Configure per-tenant Grafana dashboards
- Set up chargeback reporting for CFO validation

In [ ]:
# Cell 31: Cleanup (Clear Context)
TenantContextManager.clear_context()
print("✓ Tenant context cleared")
print("✓ Notebook complete - you are now ready to implement production multi-tenant RAG!")